In [19]:
sc.stop()

In [20]:
#Set configuration and start sparkContext again
import pyspark as ps

conf = ps.SparkConf().setMaster("local[2]").setAppName("NetworkBehavioralAnalysis")
conf.set("spark.executor.heartbeatInterval","360000s")
sc = ps.SparkContext('local[2]', '', conf=conf) 

In [21]:
import math
import itertools
import pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
from collections import OrderedDict
%matplotlib inline
gt0 = time()


In [22]:
#Columns names after one-hot-encoding
col_names=np.array(['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
       'urgent', 'hot', 'num_failed_logins', 'logged_in',
       'num_compromised', 'root_shell', 'su_attempted', 'num_root',
       'num_file_creations', 'num_shells', 'num_access_files',
       'is_host_login', 'is_guest_login', 'count', 'srv_count',
       'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
       'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate',
       'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'label', 'protocol_type_tcp',
       'protocol_type_udp', 'flag_REJ', 'flag_RSTO', 'flag_RSTOS0',
       'flag_RSTR', 'flag_S0', 'flag_S1', 'flag_S2', 'flag_S3', 'flag_SF',
       'flag_SH', 'service_X11', 'service_Z39_50', 'service_aol',
       'service_auth', 'service_bgp', 'service_courier',
       'service_csnet_ns', 'service_ctf', 'service_daytime',
       'service_discard', 'service_domain', 'service_domain_u',
       'service_echo', 'service_eco_i', 'service_ecr_i', 'service_efs',
       'service_exec', 'service_finger', 'service_ftp', 'service_ftp_data',
       'service_gopher', 'service_harvest', 'service_hostnames',
       'service_http', 'service_http_2784', 'service_http_443',
       'service_http_8001', 'service_imap4', 'service_iso_tsap',
       'service_klogin', 'service_kshell', 'service_ldap', 'service_link',
       'service_login', 'service_mtp', 'service_name',
       'service_netbios_dgm', 'service_netbios_ns', 'service_netbios_ssn',
       'service_netstat', 'service_nnsp', 'service_nntp', 'service_ntp_u',
       'service_other', 'service_pm_dump', 'service_pop_2',
       'service_pop_3', 'service_printer', 'service_private',
       'service_red_i', 'service_remote_job', 'service_rje',
       'service_shell', 'service_smtp', 'service_sql_net', 'service_ssh',
       'service_sunrpc', 'service_supdup', 'service_systat',
       'service_telnet', 'service_tftp_u', 'service_tim_i', 'service_time',
       'service_urh_i', 'service_urp_i', 'service_uucp',
       'service_uucp_path', 'service_vmnet', 'service_whois'])


In [23]:
from pyspark.mllib.regression import LabeledPoint
from numpy import array

#splited 
def parse_interaction(line):
    line_split = line.split(",")
   
    #when apply [3] method
    #symbolic_indexes = [3,5,7,8,10,11,14,16,21,23,24,27,29,34,36,37,47,48,49,50,51,52,53,56,57,58,59,60,62,65,66,67,68,70,71,72,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,99,100,101,102,104,105,106,107,108,109,110,111,112,113,114,116,117,118]
    #when apply [2] method
    #symbolic_indexes=[37,52,71,74,76,99,113]
    #when apply [1]
    symbolic_indexes=[1,2,3,41]
    clean_line_split = [item for i,item in enumerate(line_split) if i not in symbolic_indexes]
    
    attack=1
    if line_split[41]=='normal':
        attack = 0
    return LabeledPoint(attack, array([float(x) for x in clean_line_split]))

    #map integration of data set
def parse_interaction_(line):
    line_split = line.split(",")
    #symbolic_indexes = [3,5,7,8,10,11,14,16,21,23,24,27,29,34,36,37,47,48,49,50,51,52,55,56,57,58,59,61,64,65,66,67,69,70,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,95,96,97,99,100,101,102,103,104,105,106,107,108,110,111,112]  
    #symbolic_indexes=[37]
    symbolic_indexes=[1,2,3,41]
    clean_line_split = [item for i,item in enumerate(line_split) if i not in symbolic_indexes]
    
    attack=1
    if line_split[41]=='normal':
        attack = 0
    return LabeledPoint(attack, array([float(x) for x in clean_line_split]))

   

In [24]:
from pyspark.mllib.stat import Statistics
def feature_selection(data_):
    
    chi = Statistics.chiSqTest(data_)
    pd.set_option('display.max_rows', None)
   

    records = [(result.statistic, result.pValue) for result in chi]

    chi_df = pd.DataFrame(data=records, index=col_names, columns=["Statistic","p-value"])
    print(chi_df)

In [35]:
def main():
    
   
   
    url_train = r'C:\Users\erengul\Desktop\DATA_SETS\KDDTrain+.txt'
    read_train = sc.textFile(url_train).cache()

    url_test = r'C:\Users\erengul\Desktop\DATA_SETS\KDDTest+.txt'
    read_test = sc.textFile(url_test).cache()
    
    train_df=read_train.map(parse_interaction)
    test_df=read_test.map(parse_interaction_)
    
    

    from pyspark.mllib.classification import LogisticRegressionWithLBFGS
    from time import time
    from pyspark.mllib.classification import SVMWithSGD, SVMModel
    from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
    from pyspark.mllib.util import MLUtils
    from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
    from pyspark.mllib.tree import RandomForest, RandomForestModel
    
    (trainingData, testData) = train_df.randomSplit([0.7, 0.3])

    t0 = time()
    
    #just eneough to change this statment with other models as svm or others 
    model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=3, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32)
   
    # model = DecisionTree.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
    #                                 impurity='gini', maxDepth=5, maxBins=32)
    
    # model = SVMWithSGD.train(trainingData, iterations=100)
   
    #model = LogisticRegressionWithLBFGS.train(trainingData)
    
    tt = time() - t0
    print ("Classifier trained in {} seconds".format(round(tt,3)))
   
    predictions = model.predict(test_df.map(lambda p: p.features))
    labels_and_preds = test_df.map(lambda p: p.label).zip(predictions)
   

   t0 = time()
    test_accuracy = labels_and_preds.filter(lambda vp: vp[0] == vp[1]).count() / float(test_df.count())
    tt = time() - t0

    print ("Prediction made in {} seconds. Test accuracy is {}".format(round(tt,3), round(test_accuracy,4)))

In [36]:
main()

Classifier trained in 19.999 seconds
Prediction made in 34.892 seconds. Test accuracy is 0.6968


In [ ]:
spark.stop()

In [37]:

from pyspark.mllib.stat import Statistics 
correlation_matrix = Statistics.corr(vector_data, method="spearman")

In [32]:
import pandas as pd
pd.set_option('display.max_columns', 50)



corr_df = pd.DataFrame(correlation_matrix, index=col_names, columns=col_names)

print(corr_df)

                    protocol_type_tcp  protocol_type_udp  flag_REJ  flag_RSTO  \
protocol_type_tcp            1.000000          -0.772070  0.148967   0.053347   
protocol_type_udp           -0.772070           1.000000 -0.115013  -0.041188   
flag_REJ                     0.148967          -0.115013  1.000000  -0.035062   
flag_RSTO                    0.053347          -0.041188 -0.035062   1.000000   
flag_RSTOS0                  0.013619          -0.010515 -0.008951  -0.003206   
flag_RSTR                    0.066646          -0.051455 -0.043802  -0.015686   
flag_S0                      0.294442          -0.227330 -0.193520  -0.069302   
flag_S1                      0.025665          -0.019815 -0.016868  -0.006041   
flag_S2                      0.015124          -0.011677 -0.009940  -0.003560   
flag_S3                      0.009392          -0.007251 -0.006173  -0.002211   
flag_SF                     -0.392864           0.303318 -0.379183  -0.135790   
flag_SH                     

In [25]:
# get a boolean dataframe where true means that a pair of variables is highly correlated
highly_correlated_df = (abs(corr_df) > .8) & (corr_df < 1.0)
# get the names of the variables so we can use them to slice the dataframe
correlated_vars_index = (highly_correlated_df==True).any()
correlated_var_names = correlated_vars_index[correlated_vars_index==True].index
# slice it
highly_correlated_df.loc[correlated_var_names,correlated_var_names]

,src_bytes,dst_bytes,logged_in,is_guest_login,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,flag_REJ,flag_S0,flag_SF,service_ftp
src_bytes,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
dst_bytes,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
logged_in,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
is_guest_login,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
serror_rate,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,True,False,False,False,True,False,False
srv_serror_rate,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,True,False,False
rerror_rate,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False
srv_rerror_rate,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False
same_srv_rate,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False
diff_srv_rate,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False


""
